# CNN Training 

Test CNN training vs simulated dataset size.  This uses a nearly-identical copy
of Ulascan's code for generating the stim shots and training, just wrapped in
some functions to make it easier to iterate over hyperparameters.  

In [1]:
from copy import deepcopy
import time 
import os
import json 
import itertools


import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from keras.optimizers import Adam
import pymatching

from circuit_generators import *
from sampling_functions import *
from bitpack import pack_bits, unpack_bits
from circuit_partition import *
from utilities_tf import *
from CNNModel import *

2024-06-24 11:11:12.285399: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-06-24 11:11:12.285436: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-06-24 11:11:14.159215: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-06-24 11:11:14.159433: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-06-24 11:11:14.159447: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

## Surface code definitions
These are used as globals in the stim and training functions below.  (I should
update those functions to take these SC parameters as inputs, but this works
for now.) 

In [2]:
# Number of worker nodes
n_worker_nodes = 7

# Surface code specifications
d = 5
r = 2
kernel_size = 3

# Error probabilities
p = 0.01
before_round_data_depolarization = p
after_reset_flip_probability = p
after_clifford_depolarization = p
before_measure_flip_probability = p

use_rotated_z = True
observable_type = "ZL" if use_rotated_z else "XL"

# Bit types
binary_t = np.int8 # Could use even less if numpy allowed

# Measurement index type
idx_t = np.int8
n_all_measurements = r*(d**2-1) + d**2
if n_all_measurements > np.iinfo(idx_t).max:
  idx_t = np.int16
if n_all_measurements > np.iinfo(idx_t).max:
  idx_t = np.int32
if n_all_measurements > np.iinfo(idx_t).max:
  idx_t = np.int64
if n_all_measurements > np.iinfo(idx_t).max:
  idx_t = np.int128
if n_all_measurements > np.iinfo(idx_t).max:
  idx_t = np.int256
if n_all_measurements > np.iinfo(idx_t).max:
  raise RuntimeError("idx_t is too small.")

# Call signature for circuit_partition::group_det_bits_kxk
call_group_det_bits_kxk = lambda det_bits_dxd, data_bits_dxd=None, d=d, r=r, k=kernel_size, use_rotated_z=use_rotated_z, binary_t=binary_t, idx_t=idx_t: group_det_bits_kxk(det_bits_dxd, d, r, k, use_rotated_z, data_bits_dxd, binary_t, idx_t)

## stim and CNN training 

In [3]:
def split_measurements(measurements, d):
  n_measurements = idx_t(measurements.shape[1])
  # Measurements on data qubits come last
  exclude_indices = np.array([-x-1 for x in range(d**2)], dtype=idx_t)
  exclude_indices = exclude_indices + n_measurements
  # Out of all measurements on data qubits, the logical qubit measurements are those on the boundary of the lattice.
  # All other equivalent X_L/Z_L operators can be found through the combination of ancilla measurements and the chosen data qubits giving us the logical qubit.
  exclude_indices_obsL = np.array([-x-1 for x in range(d*(d-1), d**2)], dtype=idx_t)
  exclude_indices_obsL = exclude_indices_obsL + n_measurements
  # From obs_bits, we want to exclude all measurements except those listed in exclude_indices_obsL
  exclude_indices_obs = np.arange(0, n_measurements, 1, dtype=idx_t)
  exclude_indices_obs = np.delete(exclude_indices_obs, exclude_indices_obsL)

  det_bits = measurements
  det_bits = np.delete(det_bits, exclude_indices, axis=1)
  obs_bits = measurements
  obs_bits = np.delete(obs_bits, exclude_indices_obs, axis=1)

  data_bits = measurements[:, exclude_indices]

  # Reverse the order of data_bits because exclude_indices starts from the last data qubit measurement, not the first
  data_bits = np.flip(data_bits, axis=1)

  return det_bits, obs_bits, data_bits

In [4]:
def get_stim_output(n_total, split):
  # this is using globals defined in the first cell!
  n_test = int(n_total*split)
  n_train = int(n_total*(1-split))
  n_samples = n_test + n_train
  decoders = ['pymatching']
  test_circuit = get_builtin_circuit(
    "surface_code:rotated_memory_"+('z' if use_rotated_z else 'x'),
    distance=d,
    rounds=r,
    before_round_data_depolarization = before_round_data_depolarization,
    after_reset_flip_probability = after_reset_flip_probability,
    after_clifford_depolarization = after_clifford_depolarization,
    before_measure_flip_probability = before_measure_flip_probability
  )

  kernel_circuit_extra_depol1 = [
    [
      f"DEPOLARIZE1({after_clifford_depolarization}) 13 25",
      f"DEPOLARIZE1({after_clifford_depolarization}) 5 13 17 19 25",
      f"#DEPOLARIZE1({after_clifford_depolarization})",
      f"DEPOLARIZE1({after_clifford_depolarization}) 12 15 19",
    ], # parity = (1, 1)
    [
      f"DEPOLARIZE1({after_clifford_depolarization}) 8 13 25",
      f"DEPOLARIZE1({after_clifford_depolarization}) 5 13 17 19 25",
      f"DEPOLARIZE1({after_clifford_depolarization}) 1 14 15",
      f"DEPOLARIZE1({after_clifford_depolarization}) 12 14 15 19",
    ], # parity = (0, 1)
    [
      f"DEPOLARIZE1({after_clifford_depolarization}) 8 25",
      f"DEPOLARIZE1({after_clifford_depolarization}) 17 25",
      f"DEPOLARIZE1({after_clifford_depolarization}) 1 14 15",
      f"DEPOLARIZE1({after_clifford_depolarization}) 14 15",
    ], # parity = (-1, 1)
    [
      f"DEPOLARIZE1({after_clifford_depolarization}) 5 13 25",
      f"DEPOLARIZE1({after_clifford_depolarization}) 5 13 17 19 25",
      f"DEPOLARIZE1({after_clifford_depolarization}) 2 3",
      f"DEPOLARIZE1({after_clifford_depolarization}) 2 12 15 19",
    ], # parity = (1, 0)
    [
      f"DEPOLARIZE1({after_clifford_depolarization}) 1 5 8 13 25",
      f"DEPOLARIZE1({after_clifford_depolarization}) 5 13 17 19 25",
      f"DEPOLARIZE1({after_clifford_depolarization}) 1 2 3 14 15",
      f"DEPOLARIZE1({after_clifford_depolarization}) 2 12 14 15 19",
    ] # parity = (0, 0)
  ]
  kernel_circuits = []
  for replace_args in kernel_circuit_extra_depol1:
    kernel_circuit_template = \
    f"""
  QUBIT_COORDS(1, 1) 1
  QUBIT_COORDS(2, 0) 2
  QUBIT_COORDS(3, 1) 3
  QUBIT_COORDS(5, 1) 5
  QUBIT_COORDS(1, 3) 8
  QUBIT_COORDS(2, 2) 9
  QUBIT_COORDS(3, 3) 10
  QUBIT_COORDS(4, 2) 11
  QUBIT_COORDS(5, 3) 12
  QUBIT_COORDS(6, 2) 13
  QUBIT_COORDS(0, 4) 14
  QUBIT_COORDS(1, 5) 15
  QUBIT_COORDS(2, 4) 16
  QUBIT_COORDS(3, 5) 17
  QUBIT_COORDS(4, 4) 18
  QUBIT_COORDS(5, 5) 19
  QUBIT_COORDS(4, 6) 25
  R 1 3 5 8 10 12 15 17 19
  X_ERROR({after_reset_flip_probability}) 1 3 5 8 10 12 15 17 19
  R 2 9 11 13 14 16 18 25
  X_ERROR({after_reset_flip_probability}) 2 9 11 13 14 16 18 25
  TICK
  DEPOLARIZE1({before_round_data_depolarization}) 1 3 5 8 10 12 15 17 19
  H 2 11 16 25
  DEPOLARIZE1({after_clifford_depolarization}) 2 11 16 25
  TICK
  CX 2 3 16 17 11 12 15 14 10 9 19 18
  DEPOLARIZE2({after_clifford_depolarization}) 2 3 16 17 11 12 15 14 10 9 19 18
  {replace_args[0]}
  TICK
  CX 2 1 16 15 11 10 8 14 3 9 12 18
  DEPOLARIZE2({after_clifford_depolarization}) 2 1 16 15 11 10 8 14 3 9 12 18
  {replace_args[1]}
  TICK
  CX 16 10 11 5 25 19 8 9 17 18 12 13
  DEPOLARIZE2({after_clifford_depolarization}) 16 10 11 5 25 19 8 9 17 18 12 13
  {replace_args[2]}
  TICK
  CX 16 8 11 3 25 17 1 9 10 18 5 13
  DEPOLARIZE2({after_clifford_depolarization}) 16 8 11 3 25 17 1 9 10 18 5 13
  {replace_args[3]}
  TICK
  H 2 11 16 25
  DEPOLARIZE1({after_clifford_depolarization}) 2 11 16 25
  TICK
  X_ERROR({before_measure_flip_probability}) 2 9 11 13 14 16 18 25
  MR 2 9 11 13 14 16 18 25
  X_ERROR({after_reset_flip_probability}) 2 9 11 13 14 16 18 25
  DETECTOR(0, 4, 0) rec[-4]
  DETECTOR(2, 2, 0) rec[-7]
  DETECTOR(4, 4, 0) rec[-2]
  DETECTOR(6, 2, 0) rec[-5]
  REPEAT {r-1} {{
    TICK
    DEPOLARIZE1({before_round_data_depolarization}) 1 3 5 8 10 12 15 17 19
    H 2 11 16 25
    DEPOLARIZE1({after_clifford_depolarization}) 2 11 16 25
    TICK
    CX 2 3 16 17 11 12 15 14 10 9 19 18
    DEPOLARIZE2({after_clifford_depolarization}) 2 3 16 17 11 12 15 14 10 9 19 18
    {replace_args[0]}
    TICK
    CX 2 1 16 15 11 10 8 14 3 9 12 18
    DEPOLARIZE2({after_clifford_depolarization}) 2 1 16 15 11 10 8 14 3 9 12 18
    {replace_args[1]}
    TICK
    CX 16 10 11 5 25 19 8 9 17 18 12 13
    DEPOLARIZE2({after_clifford_depolarization}) 16 10 11 5 25 19 8 9 17 18 12 13
    {replace_args[2]}
    TICK
    CX 16 8 11 3 25 17 1 9 10 18 5 13
    DEPOLARIZE2({after_clifford_depolarization}) 16 8 11 3 25 17 1 9 10 18 5 13
    {replace_args[3]}
    TICK
    H 2 11 16 25
    DEPOLARIZE1({after_clifford_depolarization}) 2 11 16 25
    TICK
    X_ERROR({before_measure_flip_probability}) 2 9 11 13 14 16 18 25
    MR 2 9 11 13 14 16 18 25
    X_ERROR({after_reset_flip_probability}) 2 9 11 13 14 16 18 25
    SHIFT_COORDS(0, 0, 1)
    DETECTOR(2, 0, 0) rec[-8] rec[-16]
    DETECTOR(2, 2, 0) rec[-7] rec[-15]
    DETECTOR(4, 2, 0) rec[-6] rec[-14]
    DETECTOR(6, 2, 0) rec[-5] rec[-13]
    DETECTOR(0, 4, 0) rec[-4] rec[-12]
    DETECTOR(2, 4, 0) rec[-3] rec[-11]
    DETECTOR(4, 4, 0) rec[-2] rec[-10]
    DETECTOR(4, 6, 0) rec[-1] rec[-9]
  }}
  X_ERROR({before_measure_flip_probability}) 1 3 5 8 10 12 15 17 19
  M 1 3 5 8 10 12 15 17 19
  DETECTOR(0, 4, 1) rec[-3] rec[-6] rec[-13]
  DETECTOR(2, 2, 1) rec[-5] rec[-6] rec[-8] rec[-9] rec[-16]
  DETECTOR(4, 4, 1) rec[-1] rec[-2] rec[-4] rec[-5] rec[-11]
  DETECTOR(6, 2, 1) rec[-4] rec[-7] rec[-14]
  OBSERVABLE_INCLUDE(0) rec[-7] rec[-8] rec[-9]
    """
    kernel_circuits.append(stim.Circuit(kernel_circuit_template))


  # Sampling for the dxd circuit
  m_sampler = test_circuit.compile_sampler(seed=12345)
  converter = test_circuit.compile_m2d_converter()
  detector_error_model = test_circuit.detector_error_model(decompose_errors=True)

  measurements = m_sampler.sample(n_samples, bit_packed=False)
  det_evts, flips = converter.convert(measurements=measurements, separate_observables=True, bit_packed=False)
  measurements = measurements.astype(binary_t)
  det_evts = det_evts.astype(binary_t)
  flips = flips.astype(binary_t)

  avg_flips = np.sum(flips.reshape(-1,), dtype=np.float32)/flips.shape[0]
  print(f"Average flip rate for the full circuit: {avg_flips}")

  # next cell
  n_measurements = idx_t(measurements.shape[1])
  det_bits, obs_bits, data_bits = split_measurements(measurements, d)

  # next cell
  det_bits_kxk_all, data_bits_kxk_all, obs_bits_kxk_all, kernel_result_translation_map = call_group_det_bits_kxk(det_bits, data_bits_dxd=data_bits)
  kernel_types = get_unique_kernel_types(kernel_size, d)
  n_kernels = det_bits_kxk_all.shape[0]

  # next cell
  kernel_result_translation_map_f = kernel_result_translation_map[:,:,1:]
  kernel_result_translation_map_b = kernel_result_translation_map[:,:,0:-1]
  kernel_result_translation_det_evts = (kernel_result_translation_map_f!=kernel_result_translation_map_b).astype(binary_t)
  kernel_result_translation_map = np.concatenate((kernel_result_translation_map, kernel_result_translation_det_evts), axis=2)

  # next cell
  det_evts_kxk_all = []
  flips_kxk_all = []
  converters_kernel = []
  for kernel_circuit in kernel_circuits:
    converters_kernel.append(kernel_circuit.compile_m2d_converter())
  for k in range(n_kernels):
    measurements_kxk = np.concatenate((det_bits_kxk_all[k], data_bits_kxk_all[k]), axis=1).astype(np.bool_)
    ik = 0
    for i, kernel_type in enumerate(kernel_types):
      if k in kernel_type[1]:
        ik = i
        break
    det_evts_kxk, flips_kxk = converters_kernel[ik].convert(measurements=measurements_kxk, separate_observables=True, bit_packed=False)
    det_evts_kxk_all.append(det_evts_kxk)
    flips_kxk_all.append(flips_kxk)
  det_evts_kxk_all = np.array(det_evts_kxk_all, dtype=binary_t)
  flips_kxk_all = np.array(flips_kxk_all, dtype=binary_t)
  del converters_kernel

  # next cell
  class_bits = flips
  features_det_bits = np.swapaxes(det_bits_kxk_all, 0, 1)
  features_det_evts = np.swapaxes(det_evts_kxk_all, 0, 1)
  features_translation_map = np.swapaxes(kernel_result_translation_map, 0, 1) # Dimensions go as [sample][kernel][cycle bits + detections (n_cycles-1)]
  features_translation_map = np.reshape(features_translation_map, (features_translation_map.shape[0], -1))
  features_final_det_evts = det_evts[:, -((d**2-1)//2):]

  return {"CNN": [[features_det_bits,
                   features_det_evts,
                   features_translation_map,
                   features_final_det_evts],
                  class_bits], 
          "pymatch": [detector_error_model, det_evts, flips]}

In [5]:
def get_pymatching_result(stim_output, idxs_test):
    detector_error_model, det_evts, flips = stim_output["pymatch"]
    pymatcher = pymatching.Matching.from_detector_error_model(detector_error_model)
    flips_pred_pym = pymatcher.decode_batch(det_evts, bit_packed_predictions=False, bit_packed_shots=False).astype(binary_t).reshape(-1,1)
    flips_pred_pym = pymatcher.decode_batch(det_evts[idxs_test,:], bit_packed_predictions=False, bit_packed_shots=False).astype(binary_t).reshape(-1,1)
    pymatch_error_rate_test = np.sum(
        (flips_pred_pym!=flips[idxs_test,:]))/flips_pred_pym.shape[0]
    return pymatch_error_rate_test

## Training parameters 

In [6]:
base = "sc-CNN-training"
run = "testing" #"in_memory"
n_totals = [10**4, 10**5] #[10**5, 10**6, 10**7]
frac_test = 0.2
batch_size = 10**3  
n_epochs = 20 # 60
n_nodes = 150
lrs = [0.005, 0.007] #[0.05, 0.005, 0.0005]

In [7]:
setup_params = {
  "d": d,
  "r": r,
  "kernel_size": kernel_size,
  "p": p,
  "before_round_data_depolarization": before_round_data_depolarization,
  "after_reset_flip_probability": after_reset_flip_probability,
  "after_clifford_depolarization": after_clifford_depolarization,
  "before_measure_flip_probability": before_measure_flip_probability,
  "use_rotated_z": use_rotated_z,
  "observable_type": observable_type,
  "runname": run,
  "n_totals": n_totals,
  "frac_test": frac_test,
  "batch_size": batch_size,
  "n_epochs": n_epochs,
  "n_nodes": n_nodes,
  "lrs": lrs,
}

setupfile_path = f"{base}/{run}/setup.json"
if not os.path.exists(setupfile_path):
    os.makedirs(f"{base}/{run}", exist_ok=True)
with open(setupfile_path, 'w') as f: json.dump(setup_params, f, indent=2)

## Run stim and pymatching

In [8]:
# run stim for largest desired sample size
max_total = np.max(n_totals)
stim_output = get_stim_output(max_total, frac_test)
[[features_det_bits,
  features_det_evts,
  features_translation_map,
  features_final_det_evts], class_bits] = stim_output["CNN"]

# PyMatching error rate
pymatch_error_rate = get_pymatching_result(
  stim_output, np.arange(max_total, dtype=np.int32))
flit_rate = stim_output["pymatch"][2].sum()/stim_output["pymatch"][2].shape[0]
print(f"PyMatching error rate for full data set: "
      f"{100*pymatch_error_rate:0.4f}%")

# save
pymatch_file = f"{base}/{run}/pymatch_results.json"
pymatch_results = {"flip_rate": flit_rate, "error_rate": pymatch_error_rate}
with open(pymatch_file, 'w') as f: json.dump(pymatch_results, f, indent=2)

Average flip rate for the full circuit: 0.22601
PyMatching error rate for full data set: 3.1740%


## Train CNNs

This trains the models defined above, and records the training history.  

(I would like to save the actual models, but the built-in functions to do this
fail here since the CNN here is built as a custom class. It seems we need to add
code to those classes which shows Keras how to serialize the model.  That got
complicated, so for now just save the training history and we'll need to re-train
if we want these models for anything.)

In [9]:
histories = {}
t0_train = time.time()
n_models = len(n_totals)*len(lrs)
for i, (n_total, lr) in enumerate(itertools.product(n_totals, lrs)):
  # train-test split
  # grab first n_total samples from stim, and split them
  # (Make sure the data type is np.int32 below, not idx_t!)
  idxs_test, idxs_train = split_data(
      np.arange(n_total, dtype=np.int32), 
      test_size=frac_test, 
      seed=12345, shuffle=False)
  
  # package stim output
  data = {}
  for label, index in zip(["test", "train"], 
                          [idxs_test, idxs_train]):
      data[label] = [features_det_bits[index], 
                      features_det_evts[index],
                      features_translation_map[index], 
                      features_final_det_evts[index] ]
  
  # train the model
  print(f"\nTraining CNN model {i+1}/{n_models}\n"
        f"    n_total = {n_total:0.1e} samples\n" 
        f"         lr = {lr}\n"
        + "-"*28)
  model_dxd = FullCNNModel(
    observable_type, d, kernel_size, r,
    [n_nodes for _ in range(2)], npol=2,
    do_all_data_qubits=False, extended_kernel_output=True, 
    include_det_evts=True, include_last_kernel_dets=False, 
    include_last_dets=True, has_nonuniform_response=False, 
    use_translated_kernels=False
  )
  model_dxd.compile(
     optimizer=Adam(learning_rate=lr), 
     loss='binary_crossentropy', metrics=['accuracy'])
  history = model_dxd.fit(
    x=data["train"],
    y=class_bits[idxs_train,:],
    batch_size=batch_size,
    epochs=n_epochs, 
    validation_data=(data["test"], class_bits[idxs_test,:]),
    callbacks=[
      # tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,
      #                                 restore_best_weights=True),
      # tf.keras.callbacks.LearningRateScheduler(learning_rate_scheduler)
    ]
  )

  for metric, value in history.history.items():
    histories[lr, n_total, metric] = value
  
  # evaluate the model
  print(f"\nFinished training CNN model {i+1}/{n_models}\n"
        f"    n_total = {n_total:0.1e} samples\n" 
        f"         lr = {lr}")
  best_epoch = np.argmax(history.history['val_accuracy'])
  best_er_percent = 100*(1 - history.history['val_accuracy'][best_epoch])
  print(f"Best test error rate was {best_er_percent:.4f}% "
        f"in epoch {best_epoch + 1}/{n_epochs}")

# save histories
histories_file = f"{base}/{run}/histories.csv"
histories = pd.DataFrame(histories)
histories.columns.names = ["lr", "N", "metric"]
histories.to_csv(histories_file, index=False) 

t1_train = time.time()
dt = (t1_train - t0_train)/60
print(f"\nTotal training time for {n_models} models: {dt:.2f} mins")




Training CNN model 1/4
    n_total = 1.0e+04 samples
         lr = 0.005
----------------------------


2024-06-24 11:11:18.605617: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-06-24 11:11:18.606234: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-06-24 11:11:18.606325: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (rjanish-ThinkPad-X220): /proc/driver/nvidia/version does not exist


Number of unique contributions: 13
Total number of fractions: 28
Total number of phases: 62
Epoch 1/20
2/2 [==============================] - 30s 5s/step - loss: 0.6071 - accuracy: 0.6215 - val_loss: 0.7647 - val_accuracy: 0.7653
Epoch 2/20
2/2 [==============================] - 1s 426ms/step - loss: 0.6188 - accuracy: 0.7855 - val_loss: 0.5375 - val_accuracy: 0.7653
Epoch 3/20
2/2 [==============================] - 0s 351ms/step - loss: 0.5336 - accuracy: 0.7855 - val_loss: 0.5390 - val_accuracy: 0.7653
Epoch 4/20
2/2 [==============================] - 0s 350ms/step - loss: 0.5094 - accuracy: 0.7855 - val_loss: 0.5412 - val_accuracy: 0.7653
Epoch 5/20
2/2 [==============================] - 0s 352ms/step - loss: 0.5059 - accuracy: 0.7855 - val_loss: 0.5251 - val_accuracy: 0.7653
Epoch 6/20
2/2 [==============================] - 0s 345ms/step - loss: 0.4874 - accuracy: 0.7855 - val_loss: 0.5033 - val_accuracy: 0.7653
Epoch 7/20
2/2 [==============================] - 0s 349ms/step - loss

In [11]:
histories.head(3)

lr         0.005                                     0.007                     \
N         10000                                     10000                       
metric      loss accuracy  val_loss val_accuracy      loss accuracy  val_loss   
0       0.607094   0.6215  0.764707      0.76525  0.619394   0.6840  0.534548   
1       0.618781   0.7855  0.537532      0.76525  0.499662   0.7855  0.508134   
2       0.533600   0.7855  0.539016      0.76525  0.481377   0.7840  0.507273   

lr                      0.005                                     0.007  \
N                      100000                                    100000   
metric val_accuracy      loss accuracy  val_loss val_accuracy      loss   
0          0.765250  0.487629  0.77940  0.443035     0.781012  0.529056   
1          0.765250  0.412140  0.78715  0.387641     0.791812  0.434595   
2          0.765125  0.360567  0.81500  0.345060     0.836187  0.388474   

lr                                      
N                                       
metric accuracy  val_loss val_accuracy  
0       0.74655  0.456352     0.771288  
1       0.78760  0.415658     0.787975  
2       0.79890  0.369709     0.814038